In [ ]:
from sqlalchemy import create_engine
import pandas as pd
import requests
import os
import re

%load_ext nb_black

In [ ]:
engine = create_engine(
    "postgresql://postgres:{}@localhost:5432/venmo".format(
        os.environ.get("POSTGRES_PASS", "")
    )
)

In [ ]:
trans_df = pd.read_sql_query(
    "select * from transactions order by id desc limit 10000", con=engine
)
trans_df.head()

In [ ]:
users_df = pd.read_sql_query(
    "select * from users order by id desc limit 1000", con=engine
)
users_df.head()

In [ ]:
places = pd.read_csv("us_cities_states_counties.data.csv", "|").dropna()
places.head()

In [ ]:
cities = set(places["City"].apply(lambda s: s.lower()))
banned_locs = set(
    [
        "may",
        "sweet",
        "chicken",
        "early",
        "price",
        "chase",
        "rentz",
        "jacob",
        "call",
        "deposit",
        "lovely",
        "star",
        "power",
        "gas",
        "tea",
        "santa",
        "manly",
        "street",
        "blue",
        "bell",
        "park",
        "money",
        "pillow",
        "crossroads",
        "susan",
        "beaver",
        "man",
        "central",
        "rose",
    ]
)
cities = cities - banned_locs

In [ ]:
# vacations, so many actual weird town names, sports games
for row in trans_df.iterrows():
    idx, data = row
    msg = data["message"]
    tokens = re.sub(r"[^a-zA-Z ]", "", msg.lower().strip()).split(" ")
    tokens = [t for t in tokens if t != ""]
    for token in tokens:
        if token in cities:
            print(token, "-", msg)
            break

In [ ]:
# doesn't seem like these are used much
covid_words = [
    "diagnosed",
    "pneumonia",
    "fever",
    "covid",
    "isolating",
    "tested postive",
    "quarantine",
]
for row in trans_df.iterrows():
    idx, data = row
    msg = data["message"]
    tokens = re.sub(r"[^a-zA-Z ]", "", msg.lower().strip()).split(" ")
    tokens = [t for t in tokens if t != ""]
    for token in tokens:
        if token in covid_words:
            print(token, "-", msg)
            break

In [ ]:
# ~55% have pictures
for idx, row in users_df.iterrows():
    print(row["name"])
    print(row["picture_url"])

In [ ]:
# https://developers.google.com/custom-search/v1/site_restricted_api
# ...$450k for all users
requests.get(
    "https://www.googleapis.com/customsearch/v1/siterestrict?key={}&cx={}&q={}".format(
        os.environ["CUSTOM_SEARCH_API_KEY"], "f881e7a3ef3ef62ce", "Shrivu Shankar"
    )
).json()["items"]